# ETL for True Reviews
_____

In [7]:
import numpy as np
import pandas as pd

In [2]:
file = 'source/fakespot_true_rating_a.txt'
with open(file) as f:
    content = f.read()

In [3]:
to_replace = 'Translate review to English'
content = content.replace(to_replace, '')

In [4]:
content = content.split('Report')
print(len(content))
print(content[0])

404
Mommyonthego
5.0 out of 5 stars Love em
Reviewed in the United States on September 2, 2023
Color: BlackVerified Purchase
These are great. Used them on my flight. Noise canceling was fantastic, they have an audio jack so they worked with the plane itself. I really like them.
One person found this helpful
Helpful



In [5]:
content2 = []
for c in content:
    content2.append([i.strip() for i in c.split('\n') if i.strip()])
content2 = [c for c in content2 if len(c) > 0]
len(content2)              

403

In [22]:
[c[-3] for c in content2]

['These are great. Used them on my flight. Noise canceling was fantastic, they have an audio jack so they worked with the plane itself. I really like them.',
 'Update: its been slighly over a year and i use them daily for sevral hours each day. They still work perfectly!!!!!!',
 "I purchased these to replace a pair I've had for a few years that are just getting ratty looking. These are better at cancelling out the noise of my mower, and the sound quality of the music is really good! Easy to use - I mow for about 4 hours at a time and they last easily for that length of time. I just recharge them after each time I mow, so I can't really comment on the actual length of time they hold the charge.",
 'Happy with the headphones.',
 "The sound of these are fantastic. I purchased this for my daughter who has autism and struggles with noisy environment. The noise cancelation works well, there is a noticeable difference when it is turned on and off. The sound is superb! I'm planning on getting 

In [26]:
[c for c in content2 if any('image' in item for item in c)]

[['Joelle Brodeur',
  '5.0 out of 5 stars The Sleeper Brand of Ear Buds',
  'Reviewed in the United States on March 20, 2020',
  'Color: BlackVerified Purchase',
  "I have been through a few pairs of true wireless ear buds and wasn't a fan of the fit for most (one bud would fit while the other falls out). These fit great and with the different included tips, you can swap out and customize your fit for both buds or a single bud [if your ear canals differ in size like mine do].",
  "Sound is good. I'm still trying to find my happy place with adjusting my EQ app on the phone to balance the sound on this pair. Bass is great, mids and highs are crisp. Initially, these seem to be on the quiet side, but with that said, I haven't balanced out my personal sound profile on my phone.",
  "When I saw photos of this product in the reviews here [on Amazon], I wasn't impressed because it looked like cheap plastic. On top of that, it's a brand I never heard of. I found out they are far more superior i

In [6]:
# these are used instead of names
special = [
'RKYM', 'States'
]

# modify content2 to deal w/items from var special
for idx, c in enumerate(content2):
    # remove
    if special[0] in c:
        content2[idx] = [item for item in c if item != special[0]]
    # remove
    if special[1] in c:
        content2[idx] = [item for item in c if item != special[1]]
print([i for i in content2 if special[0] in i])
print([i for i in content2 if special[1] in i])

[]
[]


In [7]:
# remove 'VINE VOICE' from content2, but keep this info in a separate data structure
vine_voices = [False]*len(content2)
line = 'VINE VOICE'
for idx, c in enumerate(content2):
    if line in c:
        content2[idx] = [item for item in c if item != line]
        vine_voices[idx] = True
sum(vine_voices)

10

In [8]:
# remove images, but keep info
images = [
    'Customer image',
    'Customer imageCustomer image',
    'Customer imageCustomer imageCustomer image',
    'Customer imageCustomer imageCustomer imageCustomer image',
    'Customer imageCustomer imageCustomer imageCustomer imageCustomer image',
    'Customer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer image',
    'Customer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer image',
    'Customer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer image',
]
images_available = [False]*len(content2)
for image in images:
    for idx, c in enumerate(content2):
        if image in c:
            content2[idx] = [item for item in c if item != image]
            images_available[idx] = True
sum(images_available)

51

In [9]:
images2 = [
    'Customer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer image',
    'Customer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer imageCustomer image',
]
for image in images2:
    for idx, c in enumerate(content2):
        if image in c:
            content2[idx] = [item for item in c if item != image]
            images_available[idx] = True
sum(images_available)

53

In [10]:
# remove 'found helpful', but keep info
helpful = [ 'One person found this helpful', ] + [str(i) + ' people found this helpful' for i in range(2,501)]
found_helpful = [0]*len(content2)
for h in helpful:
    for idx, c in enumerate(content2):
        if h in c:
            content2[idx] = [item for item in c if item != h]
            num = h.split()[0].lower().strip()
            found_helpful[idx] = 1 if num=='one' else int(num)
len([i for i in found_helpful if i>0])

138

In [11]:
# remove 'Verified Purchase', but keep info
line = 'Verified Purchase'
verified_purchase = [False]*len(content2)
for idx, c in enumerate(content2):
        if line in c:
            content2[idx] = [item for item in c if item != line]
            verified_purchase[idx] = True
print(sum(verified_purchase))

106


In [12]:
# this info is already available, just remove
line = "Vine Customer Review of Free Product( What's this? )"
count = 0
for idx, c in enumerate(content2):
        if line in c:
            content2[idx] = [item for item in c if item != line]
            count += 1
print(f"Removed this {count} times")

Removed this 0 times


__PREPROCESSING SUMMARY__
* vine_voices keep vine voices
* images_available keep info about images
* found_helpful - contains info about # people who found this helpful
* verified_purchase - info about %

* i[0] - name
* i[1] - stars + title `'5.0 out of 5 stars Work great and sound great',`
* i[2] - date `'Reviewed in the United States on July 25, 2023'`
* i[3] - color or Verified Purchaser or both or vine (fix in the df): `'Color: BLACKVerified Purchase'`,
 `'Color: WhiteVerified Purchase'`, `'Color: Black'`, `'Color: White'`
* i[-1] - Helpful

In [13]:
# distribution of review lengths
set([len(i) for i in content2])

{4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 25, 27}

In [14]:
# get text by excluding other columns (-1 for Helpful)
text = ['\n'.join(i[3:-1]) for i in content2]
len(text)

403

In [15]:
df_true = pd.DataFrame({
    'text': text,
    'name': [i[0] for i in content2],
    'stars_title': [i[1] for i in content2],
    'date': [i[2] for i in content2],
    'is_vine_voice': vine_voices,
    'has_images': images_available,
    'found_helpful': found_helpful,
    'verified_purchase': verified_purchase,
})
print(df_true.shape)
df_true.head()

(403, 8)


,text,name,stars_title,date,is_vine_voice,has_images,found_helpful,verified_purchase
0,Color: BlackVerified Purchase\nThese are great...,Mommyonthego,5.0 out of 5 stars Love em,"Reviewed in the United States on September 2, ...",False,False,1,False
1,I originally got these for my wife. After seei...,William,5.0 out of 5 stars Amazing for the cost,"Reviewed in the United States on September 15,...",False,False,4,True
2,I purchased these to replace a pair I've had f...,Laurie W.,5.0 out of 5 stars Great for mowing,"Reviewed in the United States on August 29, 2022",False,False,10,True
3,Very comfortable headphones to wear. As a pers...,SavvyShopper,5.0 out of 5 stars ANC is pretty good,"Reviewed in the United States on May 28, 2022",False,False,12,True
4,The sound of these are fantastic. I purchased ...,Tom43876,5.0 out of 5 stars Fantastic Sound,"Reviewed in the United States on October 20, 2022",False,False,10,True


In [17]:
# get date, convert to datetime
line = 'Reviewed in the United States on'
df_true['date'] = df_true['date'].apply(lambda x: x.replace(line,'').strip())
df_true['date'] = df_true['date'].str.strip()
df_true['date'] = pd.to_datetime( df_true['date'] )

In [18]:
# split stars and title into separate columns
df_true[['stars', 'title']] = df_true['stars_title'].str.split('out of 5 stars', 2, expand=True)
df_true['stars'] = df_true['stars'].str.strip()
df_true['stars'] = df_true['stars'].astype(float)
df_true['stars'] = df_true['stars'].apply(lambda x: int(x))
df_true['title'] = df_true['title'].str.strip()
df_true = df_true.drop(['stars_title'], axis=1)
df_true['text'] = df_true['text'].str.strip()

/var/folders/vb/p2r9brhx2335cwnww04p9w180000gn/T/ipykernel_48312/1451221954.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df_true[['stars', 'title']] = df_true['stars_title'].str.split('out of 5 stars', 2, expand=True)


In [20]:
df_true.head()

,text,name,date,is_vine_voice,has_images,found_helpful,verified_purchase,stars,title
0,Color: BlackVerified Purchase\nThese are great...,Mommyonthego,2023-09-02,False,False,1,False,5,Love em
1,I originally got these for my wife. After seei...,William,2022-09-15,False,False,4,True,5,Amazing for the cost
2,I purchased these to replace a pair I've had f...,Laurie W.,2022-08-29,False,False,10,True,5,Great for mowing
3,Very comfortable headphones to wear. As a pers...,SavvyShopper,2022-05-28,False,False,12,True,5,ANC is pretty good
4,The sound of these are fantastic. I purchased ...,Tom43876,2022-10-20,False,False,10,True,5,Fantastic Sound


In [21]:
# reorder columns
cols = [ 'title', 'text', 'name', 'date', 'stars', 'is_vine_voice', 'has_images', 'found_helpful',
         'verified_purchase',  ]
df_true = df_true[cols]
df_true.head()

,title,text,name,date,stars,is_vine_voice,has_images,found_helpful,verified_purchase
0,Love em,Color: BlackVerified Purchase\nThese are great...,Mommyonthego,2023-09-02,5,False,False,1,False
1,Amazing for the cost,I originally got these for my wife. After seei...,William,2022-09-15,5,False,False,4,True
2,Great for mowing,I purchased these to replace a pair I've had f...,Laurie W.,2022-08-29,5,False,False,10,True
3,ANC is pretty good,Very comfortable headphones to wear. As a pers...,SavvyShopper,2022-05-28,5,False,False,12,True
4,Fantastic Sound,The sound of these are fantastic. I purchased ...,Tom43876,2022-10-20,5,False,False,10,True


In [23]:
# extract colors and more data on verified purchases
colors = {
      'Color: BLACKVerified Purchase': 'black',
      'Color: WhiteVerified Purchase': 'white',
      'Color: BlackVerified Purchase': 'black',
      'Color: Dark GreenVerified Purchase': 'dark green',
      'Color: GreenVerified Purchase': 'green',
      'Color: Earhooks-GoldVerified Purchase': 'earhooks-gold',
      'Color: Shadow GrayVerified Purchase': 'shadow gray',
      'Color: Navy BlueVerified Purchase': 'navy blue',
      'Color: Rose GoldVerified Purchase': 'rose gold',
      'Color: PurpleVerified Purchase': 'purple',
      'Color: Black-BrownVerified Purchase': 'balck-brown',
}
colors2 = [
    'Color: Black', 'Color: White',
         ]
indirect_colors = ['black color', 'red color', ]
df_true['color'] = np.nan

def exctract_color(row):
    t = row['text']        
    for k,v in colors.items():
        if k in t:
            row['text'] = t.replace(k, '').strip()
            row['color'] = v
            row['verified_purchase'] = True    
    if colors2[0] in t:
        row['text'] = t.replace(colors2[0], '').strip()
        row['color'] = 'black'
    elif colors2[1] in t:
        row['text'] = t.replace(colors2[1], '').strip()
        row['color'] = 'white'
    elif indirect_colors[0] in t:
        row['color'] = 'black'
    elif indirect_colors[1] in t:
        row['color'] = 'red'
    return row

df_true = df_true.apply( exctract_color, axis=1 )
df_true['color'].value_counts()

black            243
dark green         6
green              6
navy blue          6
shadow gray        4
purple             4
rose gold          3
earhooks-gold      2
white              2
Name: color, dtype: int64

In [25]:
# reorder columns
cols = [ 'title', 'text', 'name', 'date', 'stars', 'color', 'is_vine_voice', 'has_images', 'found_helpful',
         'verified_purchase',  ]
df_true = df_true[cols]
df_true.head()

,title,text,name,date,stars,color,is_vine_voice,has_images,found_helpful,verified_purchase
0,Love em,Verified Purchase\nThese are great. Used them ...,Mommyonthego,2023-09-02,5,black,False,False,1,True
1,Amazing for the cost,I originally got these for my wife. After seei...,William,2022-09-15,5,NaN,False,False,4,True
2,Great for mowing,I purchased these to replace a pair I've had f...,Laurie W.,2022-08-29,5,NaN,False,False,10,True
3,ANC is pretty good,Very comfortable headphones to wear. As a pers...,SavvyShopper,2022-05-28,5,NaN,False,False,12,True
4,Fantastic Sound,The sound of these are fantastic. I purchased ...,Tom43876,2022-10-20,5,NaN,False,False,10,True


In [29]:
# check if empty text - 11 has a title
df_true[ df_true['text']=='' ]

,title,text,name,date,stars,color,is_vine_voice,has_images,found_helpful,verified_purchase
11,Nothing to envy big names!!!,,Paola Lima,2022-05-23,5,NaN,False,True,0,True


In [31]:
df_true.loc[11, 'text'] = 'Nothing to envy big names!!!'

In [35]:
# text duplicates, but other elements in row are different - None
df_true.drop_duplicates()[ df_true.drop_duplicates().duplicated(subset=['text'], keep=False) ]

,title,text,name,date,stars,color,is_vine_voice,has_images,found_helpful,verified_purchase


In [36]:
# drop duplicates
print(df_true.shape)
df_true = df_true.drop_duplicates()
df_true.shape

(403, 10)


(393, 10)

In [5]:
# filter out non-English reviews and replace emojis with text
import pycld2 as cld2
import emoji

def detect_lang( s ):
    '''
        Return the language(s) in string s.
        Naive Bayes classifier under the hood -
        results are less certain for strings that are too short.
        Returns up to three languages with confidence scores.
        More on usage: https://pypi.org/project/pycld2/
    '''
    _, _, details = cld2.detect( s )
    return details[0][0]


def replace_emoji(s, to_text=True):
    '''
        Replace emojis in string s with their corresponding textual description if to_text=True;
        otherwise, delete all emojis
    '''

    # convert emojis to text
    if to_text:
        s2 = emoji.demojize( s )

    # or delete emojis
    else:
        for e in emoji.EMOJI_DATA:
            if e in s:
                s = s.replace(e, '')
    # see what's changed    
    if s2 != s:
        print('OLD TEXT:\n', s, sep='')
        print('\nNEW TEXT:\n', s2, sep='')
        print('\n', '='*75, '\n', sep='')

    return s2

In [8]:
file = 'source/df_true.pkl'
df_true = pd.read_pickle(file)
print(df_true.shape)
df_true.head()

(393, 10)


,title,text,name,date,stars,color,is_vine_voice,has_images,found_helpful,verified_purchase
0,Love em,Verified Purchase\nThese are great. Used them ...,Mommyonthego,2023-09-02,5,black,False,False,1,True
1,Amazing for the cost,I originally got these for my wife. After seei...,William,2022-09-15,5,NaN,False,False,4,True
2,Great for mowing,I purchased these to replace a pair I've had f...,Laurie W.,2022-08-29,5,NaN,False,False,10,True
3,ANC is pretty good,Very comfortable headphones to wear. As a pers...,SavvyShopper,2022-05-28,5,NaN,False,False,12,True
4,Fantastic Sound,The sound of these are fantastic. I purchased ...,Tom43876,2022-10-20,5,NaN,False,False,10,True


In [9]:
df_true['lang'] = df_true['text'].apply(detect_lang)
df_true['lang'].value_counts()

ENGLISH    390
SPANISH      3
Name: lang, dtype: int64

In [10]:
# remove non-English
df_true = df_true[ df_true['lang']=='ENGLISH' ]
df_true.shape

(390, 11)

In [11]:
# replace emoji
df_true['text'] = df_true['text'].apply(replace_emoji)

OLD TEXT:
This set of Bluetooth Headphones is easily one of the best I've ever come across, and here's why:
* Overall Impression: this headset comes in BLACK that exudes class & quality like a tuxedo or black limousine. It's shaped perfectly around my head & ear contours, it's so cool looking❗❗
* Sounds: this sub-$100 Bluetooth Headset measures up with all the more famous brands, it covers all dynamic ranges equally well from trebles to bass, I was blown away❗
* Noise Cancellation: With ANC 918 certified, this blocks away those annoying ambient noises, like crying babies on the airplane, or crowd noises on the subway, or while in office, coworkers chatting away while attempting to work❗
* > 1 BT Connectivity: I've never had one that permits my BT headset to connect to two devices at the same time! This means I can talk on my cellphone, and listen to music originating from my tablet, or any other Bluetooth-capable devices! This is amazingly convenient, as I no longer need to disconnect 

In [12]:
df_true['title'] = df_true['title'].apply(replace_emoji)

OLD TEXT:
Great headphones 🎧

NEW TEXT:
Great headphones :headphone:


OLD TEXT:
👍🏼

NEW TEXT:
:thumbs_up_medium-light_skin_tone:


OLD TEXT:
⭐️⭐️⭐️⭐️⭐️

NEW TEXT:
:star::star::star::star::star:




In [15]:
# save to pickle
file = 'source/df_true.pkl'
df_true.to_pickle(file)